In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import pandas as pd

In [2]:
MAX_LENGTH = 15

In [3]:
SOS_token = 0
EOS_token = 1


class Lang:
    '''
    class for word object, storing sentences, words and word counts.
    '''
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence,device):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [5]:
def deviceInit():
    return torch.device("cpu")

In [6]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self,device):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [7]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self,device):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [8]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self,device):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
def evaluate(encoder, decoder, input_lang,output_lang, sentence, device, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence,device)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden(device)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [10]:
def langInit():
    input_lang = Lang('eng')
    output_lang = Lang('hin')
    return input_lang,output_lang

In [11]:
def sampleInit():
    df = pd.read_csv('InferenceEngHind.csv')
    x=df.shape[0]
    pairs=[]
    for i in range(x):
        temp = []
        temp.append(str(df.iloc[i,1]))
        temp.append(str(df.iloc[i,2]))
        pairs.append(temp)
    return pairs

In [12]:
def getSamples(input_lang,output_lang,pairs):
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

In [13]:
def models_init(input_lang,output_lang,device):
    Inwords = input_lang.n_words
    Onwords = output_lang.n_words
    HIDDEN_SIZE = 512
    encoder1 = EncoderRNN(Inwords,HIDDEN_SIZE).to(device)
    attn_decoder1 = AttnDecoderRNN(HIDDEN_SIZE, Onwords, dropout_p=0.1).to(device)
    weight_path = r'encoderEngHindi.pth'
    encoder1.load_state_dict(torch.load(weight_path))
    weight_path = r'attndecoderEngHindi.pth'
    attn_decoder1.load_state_dict(torch.load(weight_path))
    return (encoder1,attn_decoder1)

In [14]:
def evaluateRandomly(encoder, decoder, i, o, device,pairs,n=10):
    for t in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0],device)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [15]:
def getevaluateStrings(encoder, decoder,i,o,device,pairs,n=10):
    result = {'eng':[],'hin':[],'eval':[]}
    for t in range(n):
        pair = random.choice(pairs)
        output_words, attentions = evaluate(encoder, decoder,i,o,pair[0], device)
        output_sentence = ' '.join(output_words)
        result['eng'].append(pair[0])
        result['hin'].append(pair[1])
        result['eval'].append(output_sentence)
    return result

In [16]:
def model_evaluation(encoder,decoder,i,o,device,pairs,n=1):
    #evaluateRandomly(encoder,decoder,n)
    return getevaluateStrings(encoder,decoder,i,o,device,pairs,n)

In [17]:
def inference(n=1):
    i,o = langInit()
    pairs = sampleInit()
    device = deviceInit()
    getSamples(i,o,pairs)
    encoder,decoder = models_init(i,o,device)
    return model_evaluation(encoder,decoder,i,o,device,pairs,n)

In [18]:
#enter n
n=5
inference(5)

{'eng': ['do you know how much strawberries cost right now ?'],
 'hin': ['क्या आप जानते हैं कि अभी स्ट्रॉबेरी की कीमत कितनी है ?'],
 'eval': ['क्या आप जानते हैं कि अभी स्ट्रॉबेरी की कीमत कितनी है ? <EOS>']}